In [1]:
import os
import glob
import scanpy as sc
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd



In [1]:
import os
import glob
import scanpy as sc
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

# Define markers for different types of cells
naive_memory_cd8_markers = ["CCR7", "IL7R", "TCF7", "SELL", "SATB1", "GPR183", "LTB", "LEF1", "S100A10"]
cytotoxic_cd8_markers = ["PRF1", "GZMA", "GZMK", "NKG7"]
exhausted_cd8_markers = ["CXCL13", "HSPB1", "IRF4", "LAYN", "GIMAP6", "HSPH1", "CXCR6", "CTLA4", "PDCD1", "LAG3", "HAVCR2", "TIGIT"]

# Previous tumor markers
tumor_markers = ["MKI67", "EPCAM"]

# New tumor markers from your list
new_tumor_markers = [
    "ABHD11", "ACTR2", "AKAP1", "AP2M1", "ARPC1A", "ARPC5", "ASAH1", "CAPZA1", "CLTC", 
    "COPE", "COPG1", "CS", "CTTN", "CYB5R3", "DYNLT1", "ENSA", "FAM120A", "FAM129B", 
    "FKBP8", "GNB1", "HEBP2", "LAD1", "LUC7L3", "MRPL54", "PGRMC1", "PLXNB2", "PPP2R1A", 
    "PRELID1", "PSENEN", "PTP4A2", "QARS", "RAB1A", "RAB25", "RAP1B", "SDC1", "SF3B1", 
    "SMAGP", "STK24", "TMEM106C", "TMEM14C", "TRAPPC5", "WDR83OS"
]

# Combine the lists
tumor_markers = tumor_markers + new_tumor_markers

# Previous CAF markers
caf_markers = ["ACTA2", "FAP", "PDGFRB"]

# New CAF markers from the study
new_caf_markers = [
    "OCM2", "TTK", "TOP2A", "DLGAP5", "CENPF", "C1orf220", "CT45A10", "KIF14", "MKI67", 
    "CEP55", "OR4C9P", "ELOVL6", "KCND3", "ACSF2", "PIMREG", "KNL1", "TUBA3FP", "HSD17B7P2", 
    "SLC27A4", "ARHGAP11A", "KRTAP9-4", "RNF122", "ANLN", "ENSG00000283098", "TMEM154", "CIAO2B", 
    "UBE2D4", "ASPM", "BUB1", "PIGZ", "ENSG00000254488", "SLC25A19", "DGCR6L", "RFLNB", "NYNRIN", 
    "ZDHHC18", "RAB3A", "B3GNT9", "GRN", "APEH", "EPAS1", "CIB1", "WTIP", "AMIGO1", "OR6B3", 
    "TEX264", "CMTM4", "PTDSS2"
]

# Combine the lists
caf_markers = caf_markers + new_caf_markers

# New GWAS markers
#gwas_markers = ["SMAD3", "INAVA"]

# Define categories and colors
categories = ['Naive/Memory CD8+', 'Cytotoxic CD8+', 'Exhausted CD8+', 'Tumor', 'CAF']
colors = {
    'Naive/Memory CD8+': 'red',
    'Cytotoxic CD8+': 'green',
    'Exhausted CD8+': 'blue',
    'Tumor': 'purple',
    'CAF': 'orange',
}

# Define directory for .h5ad files
directory = '/Users/adiallo/Desktop/Thesis/Data_Documents/All_Data/Visium_samples/Visium_Data/samples'
file_paths = glob.glob(os.path.join(directory, '*.h5ad'))

# Loop over each file path
for file_path in file_paths:
    # Read the adata file
    adata = sc.read_h5ad(file_path)
    
    # Initialize a DataFrame to store the maximum expression value for each category per cell
    cells_marker_expression = pd.DataFrame(0, index=adata.obs.index, columns=categories)

    # Update expression values for each category
    for gene_list, category in zip([naive_memory_cd8_markers, cytotoxic_cd8_markers, exhausted_cd8_markers, tumor_markers, caf_markers], categories):
        for gene in gene_list:
            if gene in adata.var_names:  # Check if the gene is present
                cells_marker_expression[category] = np.maximum(cells_marker_expression[category], adata[:, gene].X.toarray().ravel())
            else:
                print(f"Gene {gene} not found in dataset. Skipping...")

    # Determine the category with the highest expression for each cell
    cells_marker_expression['MaxCategory'] = cells_marker_expression.idxmax(axis=1)

    # Plotting
    plt.figure(figsize=(10, 8))
    for category, color in colors.items():
        category_mask = cells_marker_expression['MaxCategory'] == category
        # Convert category_mask to valid indices for adata.obsm['spatial']
        valid_indices = np.where(adata.obs_names.isin(category_mask[category_mask].index))[0]

        if len(valid_indices) > 0:
            plt.scatter(
                adata.obsm['spatial'][valid_indices, 0],
                adata.obsm['spatial'][valid_indices, 1],
                label=category, color=color, alpha=0.6
            )

    plt.xlabel('Spatial coordinate X')
    plt.ylabel('Spatial coordinate Y')

    # Move the legend outside the plot
    plt.legend(loc='center left', bbox_to_anchor=(1, 0.5), title="Cell Types")
    
    plt.savefig(f"{os.path.splitext(file_path)[0]}_cell_types_plot.png", bbox_inches='tight')
    plt.close()


Gene FAM129B not found in dataset. Skipping...
Gene MRPL54 not found in dataset. Skipping...
Gene PRELID1 not found in dataset. Skipping...
Gene PTP4A2 not found in dataset. Skipping...
Gene RAP1B not found in dataset. Skipping...
Gene C1orf220 not found in dataset. Skipping...
Gene CT45A10 not found in dataset. Skipping...
Gene OR4C9P not found in dataset. Skipping...
Gene TUBA3FP not found in dataset. Skipping...
Gene HSD17B7P2 not found in dataset. Skipping...
Gene KRTAP9-4 not found in dataset. Skipping...
Gene ENSG00000283098 not found in dataset. Skipping...
Gene ENSG00000254488 not found in dataset. Skipping...
Gene DGCR6L not found in dataset. Skipping...
Gene FAM129B not found in dataset. Skipping...
Gene MRPL54 not found in dataset. Skipping...
Gene PRELID1 not found in dataset. Skipping...
Gene PTP4A2 not found in dataset. Skipping...
Gene RAP1B not found in dataset. Skipping...
Gene C1orf220 not found in dataset. Skipping...
Gene CT45A10 not found in dataset. Skipping...
Ge

/opt/anaconda3/lib/python3.12/site-packages/anndata/_core/anndata.py:1756: UserWarning: Variable names are not unique. To make them unique, call `.var_names_make_unique`.
  utils.warn_names_duplicates("var")


Gene FAM129B not found in dataset. Skipping...
Gene MRPL54 not found in dataset. Skipping...
Gene PRELID1 not found in dataset. Skipping...
Gene PTP4A2 not found in dataset. Skipping...
Gene RAP1B not found in dataset. Skipping...
Gene C1orf220 not found in dataset. Skipping...
Gene CT45A10 not found in dataset. Skipping...
Gene OR4C9P not found in dataset. Skipping...
Gene TUBA3FP not found in dataset. Skipping...
Gene HSD17B7P2 not found in dataset. Skipping...
Gene KRTAP9-4 not found in dataset. Skipping...
Gene ENSG00000283098 not found in dataset. Skipping...
Gene ENSG00000254488 not found in dataset. Skipping...
Gene DGCR6L not found in dataset. Skipping...
Gene FAM129B not found in dataset. Skipping...
Gene MRPL54 not found in dataset. Skipping...
Gene PRELID1 not found in dataset. Skipping...
Gene PTP4A2 not found in dataset. Skipping...
Gene RAP1B not found in dataset. Skipping...
Gene C1orf220 not found in dataset. Skipping...
Gene CT45A10 not found in dataset. Skipping...
Ge